### leetcode 601. 

이 문제는 100명 이상의 사람이 방문했던 날들이 연속해서 3일 이상인 경우에 

연속된 3일 이상의 테이블 컬럼 값들만 반환하는 문제.

문제는 연속된 3일 이상의 값을 어떻게 계산할 것인지가 포인트이다.

주어진 테이블에서 id 는 auto_increment 로 1씩 늘어나고 있는 auto_increment 이고 date 역시 일 별로 증가하고 있음(`정렬 상태`).

문제 풀이의 아이디어는 id 값에 대해서 rank 함수로 순위를 작성한 후

100 명 이상이라는 조건을 걸어두고 쿼리를 했을 때,

`rank 역시 1 씩 늘어나는 상황에서 만약 연속된 id 값이 쿼리된다면` 

```python
100 명 조건으로 필터링 후 순위를 매김
```
| id | rank | id - rank |
|----|------|-----------|
|  1 |    1 |         0 |
|  2 |    2 |         0 |
|  5 |    3 |         2 |
|  6 |    4 |         2 |
|  7 |    5 |         2 |

id - rank 를 했을 때 그 차가 일정할 것(등차)을 이용해서 

consecutive 를 뽑아내는 아이디어를 바탕으로 쿼리 작성.

```python
select
id,
visit_date,
people

from (select *,
             rank() over(order by id asc) as icmt,
             (id - rank() over(order by id asc)) as consecutive_checker
      from stadium
      where people >= 100 
      ) as sub_table

where 
consecutive_checker in (select is_consecutive
                        from (select *,
                              rank() over(order by id asc) as icmt,
                              (id - rank() over(order by id asc)) as is_consecutive
                              from stadium
                              where people >= 100) as sub_table2
                        group by is_consecutive
                        having(count(*)) >= 3)

order by visit_date asc; 
```

MySQL에서 윈도우 함수 내부의 컬럼을 외부에서 바로 참조할 수 없기 때문에 쿼리가 좀 길어졌다.